In [1]:

# original resolution: 4.07672 arcsec x 3.78778 arcsec, pa: -81.4035 deg 
# matching kernel: 4.65325 arcsec x 4.40231 arcsec, pa: 8.59652 deg 
# target resolution: 6 arcsec x 6 arcsec, pa: 0 deg
import numpy as np
from astropy.modeling.models import Gaussian2D
import astropy.units as u
from photutils.psf import create_matching_kernel, TopHatWindow
from astropy.io import fits
from astropy.wcs import WCS
import matplotlib.pyplot as plt
from astropy.convolution import convolve


In [2]:
filenameHC3Nvlsr = 'B5-NOEMA+30m-H3CN-10-9_cut_K_1G_fitparams_filtered_Vlsr'
filenameNH3vlsr = '../../B5_wide_multiple/data/B5_VLA_GBT_model_vc_QA'
maskfile = 'B5-NOEMA+30m-H3CN-10-9_cut_K_mask'
filename_convolved = filenameHC3Nvlsr + '_conv_NH3'

In [3]:
# we first need to transform all angle units to pixels
headerHC3N = fits.getheader(filenameHC3Nvlsr+'.fits')
headerNH3 = fits.getheader(filenameNH3vlsr+'.fits')
wcsHC3N = WCS(headerHC3N)

HC3Nvlsr = fits.getdata(filenameHC3Nvlsr+'.fits')
mask = fits.getdata(maskfile+'.fits')
kernelsize = 29
xcent = int(round(kernelsize/2))

pixsizeHC3N = np.abs(headerHC3N['CDELT2']) # pixel size in degrees
bmaj_original = headerHC3N['BMAJ']
bmin_original = headerHC3N['BMIN']
bpa_original = headerHC3N['BPA'] * u.deg
sigmamaj_original_pix = (bmaj_original/pixsizeHC3N) / np.sqrt(8*np.log(2))
sigmamin_original_pix = (bmin_original/pixsizeHC3N) / np.sqrt(8*np.log(2))

bmaj_target = headerNH3['BMAJ']
bmin_target = headerNH3['BMIN']
bpa_target = headerNH3['BPA'] * u.deg
sigmamaj_target_pix = (bmaj_target/pixsizeHC3N) / np.sqrt(8*np.log(2))
sigmamin_target_pix = (bmin_target/pixsizeHC3N) / np.sqrt(8*np.log(2))

In [4]:
y, x = np.mgrid[0:kernelsize, 0:kernelsize]
beamoriginalg = Gaussian2D(100, xcent, xcent, sigmamaj_original_pix, sigmamin_original_pix, theta=bpa_original)
beamtargetg = Gaussian2D(100, xcent, xcent, sigmamaj_target_pix, sigmamin_target_pix, theta=bpa_target)
beamoriginal = beamoriginalg(x, y)
beamtarget = beamtargetg(x, y)
beamtarget /= np.sum(beamtarget)

In [5]:
window = TopHatWindow(beta=0.45)
matchingkernel = create_matching_kernel(beamoriginal, beamtarget, window=window)

In [6]:
HC3Nvlsrconvolved = convolve(HC3Nvlsr, matchingkernel)
HC3Nvlsrconvolved = np.where(mask, HC3Nvlsrconvolved, np.nan)

In [9]:
newheaderHC3N = headerHC3N.copy()
newheaderHC3N['BMAJ'] = bmaj_target
newheaderHC3N['BMIN'] = bmin_target
newheaderHC3N['BPA'] = bpa_target.value

fits.writeto(filename_convolved+'.fits', HC3Nvlsrconvolved, newheaderHC3N)